In [ ]:
# 구글 드라이브 연동
from google.colab import drive
drive.mount("/content/drive")

# GPU 설정 확인
#!nvidia-smi

# 라이브러리
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf

# imdb 데이터
vocab = 100000                                  # 단어 사전의 크기 설정
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words = vocab)

# 데이터 처리
length = []                                     # 각 리뷰의 단어 개수 확인
for data in X_train:
    length.append(len(data))
print(f'학습용 개별 리뷰의 길이: \n{length}')

plt.hist(length, bins = 50)                     # 학습용 데이터의 리뷰의 길이 분포 시각화
plt.xlabel('length of review')
plt.ylabel('number of review')
plt.show()
print(np.percentile(length, 90))                # 90%의 해당 길이 확인

maxlen = 470                                    # 90%의 해당 길이로 설정
X_train_pad = tf.keras.utils.pad_sequences(X_train, maxlen = maxlen)
X_test_pad = tf.keras.utils.pad_sequences(X_test, maxlen = maxlen)
print([len(sequence) for sequence in X_train_pad[:5]])

label_train = pd.Series(y_train).value_counts() # 레이블 분포의 편향성 확인
label_test = pd.Series(y_test).value_counts()
print(label_train)
print(label_test)

# LSTM model 생성
maxlen = 470
lstm_model = tf.keras.Sequential()
lstm_model.add(tf.keras.layers.Embedding(input_dim = 10000, output_dim = 32, input_length = maxlen))
lstm_model.add(tf.keras.layers.LSTM(16))
lstm_model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))
lstm_model.summary()

# 모델 컴파일
lstm_model.compile(loss = 'binary_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])

# 매개변수 설정하기
filepath = '/content/drive/MyDrive/NLP/lstm.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
                                                save_best_only= True,
                                                monitor = 'val_loss')
earlystop =  tf.keras.callbacks.EarlyStopping(patience= 5,  monitor='val_loss')

# 학습 진행
lstm_model.fit(x = X_train_pad,
              y = y_train,
              batch_size = 256,
              epochs = 10,
              validation_split = 0.25,
              callbacks = [earlystop, checkpoint])

# 평가하기
result = lstm_model.evaluate(X_test_pad, y_test)
print(result)

# 학습 모델 재사용
file_path = '/content/drive/MyDrive/NLP/lstm.h5'
lstm_model.load_weight(file_path)